In [1]:
import os, pathlib

file_path = os.path.dirname(os.path.realpath("__file__"))
home_dir = pathlib.Path(file_path).parent.parent
os.chdir(home_dir)

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from src.utils import SAVED_PICS_PATH, keras_dataset_image_preprocessing, DENSE_LAYER_NAME, CONV_LAYER_NAME, ALPHAS
from src.utils import SAVED_CLASSIFIER_PATH_MNIST, SAVED_CLADEC_PATH_MNIST, SAVED_REFAE_PATH_MNIST
from src.cladec import ClaDec, RefAE

(train_images, train_labels_c), (test_images, test_labels_c) = keras_dataset_image_preprocessing(keras.datasets.mnist)
SAVED_EVAL_PICS_PATH = SAVED_PICS_PATH.joinpath('fashion_mnist')

tf.random.set_seed(1234)
np.random.seed(1234)

In [3]:
classifier = keras.models.load_model(SAVED_CLASSIFIER_PATH_MNIST)
classifier.evaluate(test_images, test_labels_c)

2022-07-26 21:25:35.413759: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 21:25:35.445823: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 21:25:35.446133: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 21:25:35.447117: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

313/313 [==============================] - 2s 4ms/step - loss: 0.0375 - categorical_accuracy: 0.9903


[0.037522297352552414, 0.9902999997138977]

In [4]:
decoder_dense = ClaDec.create_128_dense_decoder()
for alpha in ALPHAS:
    cladec_path = SAVED_CLADEC_PATH_MNIST.joinpath(f'{alpha:2}')
    cladec_dense = ClaDec.create_from_weights(cladec_path.joinpath(DENSE_LAYER_NAME).joinpath(DENSE_LAYER_NAME),
                                              classifier,
                                              DENSE_LAYER_NAME,
                                              alpha / 100,
                                              decoder_dense)
    print("Evaluating ", cladec_path.joinpath(DENSE_LAYER_NAME))
    cladec_dense.evaluate(test_images, test_labels_c)

Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 0/my_dense
313/313 [==============================] - 2s 6ms/step - loss: 20.7040 - reconstruction_loss: 20.5223 - classification_loss: 0.1617
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 1/my_dense
313/313 [==============================] - 2s 6ms/step - loss: 20.5799 - reconstruction_loss: 20.5715 - classification_loss: 0.1537
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 5/my_dense
313/313 [==============================] - 2s 6ms/step - loss: 19.7239 - reconstruction_loss: 20.6083 - classification_loss: 0.1544
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/10/my_dense
313/313 [==============================] - 2s 6ms/step - loss: 18.6326 - reconstruction_loss: 20.5263 - classification_loss: 0.1495
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/25/my_dense
313/313 [==============================] - 2s 6ms/step - loss: 15.6

In [5]:
decoder_conv = ClaDec.create_64_conv_decoder()
for alpha in ALPHAS:
    cladec_path = SAVED_CLADEC_PATH_MNIST.joinpath(f'{alpha:2}')
    cladec_conv = ClaDec.create_from_weights(cladec_path.joinpath(CONV_LAYER_NAME).joinpath(CONV_LAYER_NAME),
                                             classifier,
                                             CONV_LAYER_NAME,
                                             alpha / 100,
                                             decoder_conv)
    print("Evaluating ", cladec_path.joinpath(CONV_LAYER_NAME))
    cladec_conv.evaluate(test_images, test_labels_c)

Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 0/my_conv
313/313 [==============================] - 2s 6ms/step - loss: 1.4723 - reconstruction_loss: 1.4671 - classification_loss: 0.0524
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 1/my_conv
313/313 [==============================] - 2s 6ms/step - loss: 1.4776 - reconstruction_loss: 1.4881 - classification_loss: 0.0533
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/ 5/my_conv
313/313 [==============================] - 2s 6ms/step - loss: 1.4090 - reconstruction_loss: 1.4774 - classification_loss: 0.0519
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/10/my_conv
313/313 [==============================] - 2s 6ms/step - loss: 1.3146 - reconstruction_loss: 1.4517 - classification_loss: 0.0549
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/cladec/25/my_conv
313/313 [==============================] - 2s 6ms/step - loss: 1.1127 - reconstr

In [6]:
# load RefAE model
refae_dense = RefAE.create_from_weights(SAVED_REFAE_PATH_MNIST.joinpath(DENSE_LAYER_NAME).joinpath(DENSE_LAYER_NAME),
                                        cladec_dense)
refae_conv = RefAE.create_from_weights(SAVED_REFAE_PATH_MNIST.joinpath(CONV_LAYER_NAME).joinpath(CONV_LAYER_NAME),
                                       cladec_conv)
# get reference reconstructions
print("Evaluating ", SAVED_REFAE_PATH_MNIST.joinpath(DENSE_LAYER_NAME))
refae_dense.evaluate(test_images)
print("Evaluating ", SAVED_REFAE_PATH_MNIST.joinpath(CONV_LAYER_NAME))
refae_conv.evaluate(test_images)

Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/refae/my_dense
313/313 [==============================] - 1s 3ms/step - reconstruction_loss: 3.1312
Evaluating  /home/tom/Uni_Dev/cladec-mnist/saved-models/mnist/refae/my_conv
313/313 [==============================] - 1s 3ms/step - reconstruction_loss: 0.3595


0.3594546616077423

# Train classifier on reconstructions

In [7]:
from src.create_models import train_classifier

train_reconstructions_refae_dense = refae_dense.predict(train_images)
train_reconstructions_cladec_dense = cladec_dense.predict(train_images)

test_reconstructions_refae_dense = refae_dense.predict(test_images)
test_reconstructions_cladec_dense = cladec_dense.predict(test_images)

313/313 [==============================] - 1s 3ms/step


In [8]:
classifier_refae_dense = train_classifier(train_reconstructions_refae_dense, train_labels_c, 1)

1875/1875 [==============================] - 13s 6ms/step - loss: 0.1757 - categorical_accuracy: 0.9463


In [9]:
classifier_refae_dense.evaluate(test_images, test_labels_c)

313/313 [==============================] - 1s 3ms/step - loss: 0.0681 - categorical_accuracy: 0.9787


[0.06811708211898804, 0.9786999821662903]

In [10]:
classifier_refae_dense.evaluate(test_reconstructions_refae_dense, test_labels_c)

313/313 [==============================] - 1s 3ms/step - loss: 0.0826 - categorical_accuracy: 0.9742


[0.08261319249868393, 0.9742000102996826]

In [11]:
classifier_cladec_99_dense = train_classifier(train_reconstructions_cladec_dense, train_labels_c, 1)

1875/1875 [==============================] - 12s 6ms/step - loss: 0.0763 - categorical_accuracy: 0.9784


In [12]:
classifier_cladec_99_dense.evaluate(test_images, test_labels_c)

313/313 [==============================] - 1s 3ms/step - loss: 0.4821 - categorical_accuracy: 0.9126


[0.4821401834487915, 0.9125999808311462]

In [13]:
classifier_cladec_99_dense.evaluate(test_reconstructions_cladec_dense, test_labels_c)

313/313 [==============================] - 1s 4ms/step - loss: 0.0606 - categorical_accuracy: 0.9854


[0.06057608500123024, 0.9854000210762024]